In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import sklearn
import seaborn as sns
from sklearn import metrics
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
%matplotlib inline
style.use("ggplot")

accuracy = metrics.accuracy_score

/Users/kerrylam/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("titanic_training.csv", header = 0) 
test = pd.read_csv("titanic_test.csv", header = 0)
ID = test['PassengerId'] # Extract ID Names From Test Set
data = pd.concat([train, test], axis = 0)
print(data.shape)
# data.head()

(1309, 12)


In [3]:
cols = data.columns.tolist()
cols.insert(0, cols.pop(cols.index('Survived')))
data = data.reindex(columns= cols)
data.head()

,Survived,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Ticket
0,0.0,22.0,0,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,A/5 21171
1,1.0,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,PC 17599
2,1.0,26.0,0,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,STON/O2. 3101282
3,1.0,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,113803
4,0.0,35.0,0,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,373450


### Preprocess

In [4]:
for char in data:
    data["Cabin"] = data["Cabin"].str[0]

In [5]:
# data.Cabin.value_counts()

In [6]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(np.array(data.Cabin))
# data["Cabin"] = le.transform(data.Cabin)

In [7]:
def transform(char): 
    if char == "0":
        return 5 
    if char == "A":
        return 8 
    if char == "B":
        return 7 
    if char == "C":
        return 6 
    if char == "D":
        return 5 
    if char == "E":
        return 4 
    if char == "F":
        return 3 
    if char == "G":
        return 2 
    if char == "T":
        return 1

data['Cabin'] = data['Cabin'].apply(transform)

In [8]:
def transform(room): 
    if room <= 2: return "Upper Class"
    if room > 2 and room <= 5: return "Middle Class"
    if room > 5: return "Lower Class"

data['Cabin'] = data['Cabin'].apply(transform)
data.head()

,Survived,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Ticket
0,0.0,22.0,Middle Class,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,A/5 21171
1,1.0,38.0,Lower Class,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,PC 17599
2,1.0,26.0,Middle Class,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,STON/O2. 3101282
3,1.0,35.0,Lower Class,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,113803
4,0.0,35.0,Middle Class,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,373450


In [9]:
data['Title'] = 'Other'
data.loc[['Master.' in title for title in data.Name], 'Title'] = 'Master'
data.loc[['Miss.' in title for title in data.Name], 'Title'] = 'Miss'
data.loc[['Mr.' in title for title in data.Name], 'Title'] = 'Mr'
data.loc[['Mrs.' in title for title in data.Name], 'Title'] = 'Mrs'
data.head()

,Survived,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Ticket,Title
0,0.0,22.0,Middle Class,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,A/5 21171,Mr
1,1.0,38.0,Lower Class,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,PC 17599,Mrs
2,1.0,26.0,Middle Class,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,STON/O2. 3101282,Miss
3,1.0,35.0,Lower Class,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,113803,Mrs
4,0.0,35.0,Middle Class,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,373450,Mr


In [10]:
# Fill Missing Age With Median Age Based On The Median Of A Person's Title
data["Age"].fillna(data.groupby("Title")["Age"].transform("mean"), inplace=True)

In [11]:
# le.fit(np.array(data.Title))
# data["Title"] = le.transform(data.Title)
data['Sex'] = data.Sex.map({'male':1, 'female':0})

In [12]:
data = pd.get_dummies(data, columns = ['Sex', 'Embarked', 'Title', 'Cabin'], drop_first = True)
data.head()

,Survived,Age,Fare,Name,Parch,PassengerId,Pclass,SibSp,Ticket,Sex_1,Embarked_C,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class
0,0.0,22.0,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,A/5 21171,1,0,0,1,0,1,0,0,1,0
1,1.0,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,PC 17599,0,1,0,0,0,0,1,0,0,0
2,1.0,26.0,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,STON/O2. 3101282,0,0,0,1,1,0,0,0,1,0
3,1.0,35.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,113803,0,0,0,1,0,0,1,0,0,0
4,0.0,35.0,8.0500,"Allen, Mr. William Henry",0,5,3,0,373450,1,0,0,1,0,1,0,0,1,0


In [13]:
del data["Embarked_C"]
data.head()

,Survived,Age,Fare,Name,Parch,PassengerId,Pclass,SibSp,Ticket,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class
0,0.0,22.0,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,A/5 21171,1,0,1,0,1,0,0,1,0
1,1.0,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,PC 17599,0,0,0,0,0,1,0,0,0
2,1.0,26.0,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,STON/O2. 3101282,0,0,1,1,0,0,0,1,0
3,1.0,35.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,113803,0,0,1,0,0,1,0,0,0
4,0.0,35.0,8.0500,"Allen, Mr. William Henry",0,5,3,0,373450,1,0,1,0,1,0,0,1,0


### Sibling/Spouse

In [14]:
def transform(num): 
    if num < 1: return "ByYourself"
    if num > 1: return "Has1Family"
    else: return "HasMultipleFamily"

data['SibSp'] = data['SibSp'].apply(transform)
data.head()

,Survived,Age,Fare,Name,Parch,PassengerId,Pclass,SibSp,Ticket,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class
0,0.0,22.0,7.2500,"Braund, Mr. Owen Harris",0,1,3,HasMultipleFamily,A/5 21171,1,0,1,0,1,0,0,1,0
1,1.0,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,HasMultipleFamily,PC 17599,0,0,0,0,0,1,0,0,0
2,1.0,26.0,7.9250,"Heikkinen, Miss. Laina",0,3,3,ByYourself,STON/O2. 3101282,0,0,1,1,0,0,0,1,0
3,1.0,35.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,HasMultipleFamily,113803,0,0,1,0,0,1,0,0,0
4,0.0,35.0,8.0500,"Allen, Mr. William Henry",0,5,3,ByYourself,373450,1,0,1,0,1,0,0,1,0


In [15]:
data = pd.get_dummies(data, columns = ['SibSp'], drop_first = True)

### Age

In [16]:
def transform(num): 
    if num <= 10: return "Young"
    if num > 10 and num <= 30: "Mid"
    if num > 30: return "Old"

data['Age'] = data['Age'].apply(transform)
data.head()

,Survived,Age,Fare,Name,Parch,PassengerId,Pclass,Ticket,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,SibSp_Has1Family,SibSp_HasMultipleFamily
0,0.0,None,7.2500,"Braund, Mr. Owen Harris",0,1,3,A/5 21171,1,0,1,0,1,0,0,1,0,0,1
1,1.0,Old,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,PC 17599,0,0,0,0,0,1,0,0,0,0,1
2,1.0,None,7.9250,"Heikkinen, Miss. Laina",0,3,3,STON/O2. 3101282,0,0,1,1,0,0,0,1,0,0,0
3,1.0,Old,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,113803,0,0,1,0,0,1,0,0,0,0,1
4,0.0,Old,8.0500,"Allen, Mr. William Henry",0,5,3,373450,1,0,1,0,1,0,0,1,0,0,0


In [17]:
data = pd.get_dummies(data, columns = ['Age'], drop_first = True)

### Fare

In [18]:
def transform(num): 
    if num <= 20: return 0
    else: return 1

data['Fare'] = data['Fare'].apply(transform)
data.head()

,Survived,Fare,Name,Parch,PassengerId,Pclass,Ticket,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,SibSp_Has1Family,SibSp_HasMultipleFamily,Age_Young
0,0.0,0,"Braund, Mr. Owen Harris",0,1,3,A/5 21171,1,0,1,0,1,0,0,1,0,0,1,0
1,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,PC 17599,0,0,0,0,0,1,0,0,0,0,1,0
2,1.0,0,"Heikkinen, Miss. Laina",0,3,3,STON/O2. 3101282,0,0,1,1,0,0,0,1,0,0,0,0
3,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,113803,0,0,1,0,0,1,0,0,0,0,1,0
4,0.0,0,"Allen, Mr. William Henry",0,5,3,373450,1,0,1,0,1,0,0,1,0,0,0,0


In [19]:
# Fill Missing Fare With Median Fare Of Each Pclass
data["Fare"].fillna(data.groupby("Pclass")["Fare"].transform("mean"), inplace=True)

In [20]:
data = pd.get_dummies(data, columns = ['Fare'], drop_first = True)

In [21]:
data = pd.get_dummies(data, columns = ['Pclass'], drop_first = True)

In [22]:
del data['Name'], data['Ticket'], data['PassengerId']

In [23]:
data.head()

,Survived,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,SibSp_Has1Family,SibSp_HasMultipleFamily,Age_Young,Fare_1,Pclass_2,Pclass_3
0,0.0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
1,1.0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0
2,1.0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1
3,1.0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0
4,0.0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1


### Convert DataFrame to Numeric

In [24]:
data = data.fillna(data.mean(), inplace = True) 
data = data.astype(float) # Convert DF Type to Float

"""Split Data"""
train = data[0:len(train)]
test = data[len(train):]

"""Normalize Data for Faster Computation"""
train = train/train.max().astype(np.float64)
test = test/test.max().astype(np.float64)

In [25]:
# print("Index Location of Target:", data.columns.get_loc("Survived"))

In [26]:
# train = pd.DataFrame(train.ix[:,1:])
# target = train["Survived"]

del test['Survived'] 

In [27]:
print(train.shape)
print(test.shape)

(891, 17)
(418, 16)


In [28]:
train.head()

,Survived,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,SibSp_Has1Family,SibSp_HasMultipleFamily,Age_Young,Fare_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Part 1: 

Partition Training Data into 5 Folds:

- Create train_meta with same rows and IDs as training data
- Create Empty Columns: M1 and M2 (NaNs)

- Create test_meta variable with same rows and IDs as test dataset 
- Create Empty Columns: M1 and M2 (NaNs)

In [29]:
"""Create Our Train and Test Meta Sets"""
train_meta = pd.DataFrame.copy(train)
train_meta["M1"] = np.nan
train_meta["M2"] = np.nan
train_meta["M3"] = np.nan
train_meta["M4"] = np.nan
train_meta["M5"] = np.nan
train_meta["M6"] = np.nan
print(train.shape)
train_meta.head()

(891, 17)


,Survived,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,...,Age_Young,Fare_1,Pclass_2,Pclass_3,M1,M2,M3,M4,M5,M6
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
test_meta = pd.DataFrame.copy(test)
print(test.shape)
test_meta.head()

(418, 16)


,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,SibSp_Has1Family,SibSp_HasMultipleFamily,Age_Young,Fare_1,Pclass_2,Pclass_3
0,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.111111,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


### Part 2

For each train fold: combine the other 4 into a training set and predict on each fold i 
- train fold 1 = 2,3,4,5 (Predict on fold 1)
- train fold 2 = 1,3,4,5 (Predict on Fold 2)
- train fold 3 = 1,2,4,5 (Predict on Fold 3)
- train fold 4 = 1,2,3,5 (Predict on Fold 4)
- train fold 5 = 1,2,3,4 (Predict on Fold 5)

In [31]:
"""This Data Will Be Stored In The Code At The Bottom"""
from sklearn.model_selection import KFold
kf = KFold(n_splits = 15, shuffle = False)
for train_index, test_index in kf.split(train):
    print("Train:", train_index, "Test:", test_index, "\n")

Train: [ 60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239
 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257
 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275
 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293
 294 295 296 297 298 299 300 301 302 303 304

In [32]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 0


In [33]:
j = 1
# The Variable "j" Sets It So That Our First Training Set Will Be Called: train_fold1, test_fold1, target_fold1, etc..

for train_index, test_index in kf.split(train):
# Set Up To {1-10} Train Folds With All The Data
    print ('train_fold{} = train'.format(j))  
    
# Seperate Up To {1-10} Test Folds Without The Class
    print ('test_fold{} = train.ix[{}:{}, 1:]'.format(j, min(test_index), max(test_index) + 1))
    
# Seperate Up To {1-10} Target Folds Using Only The Class
    print ('target_fold{} = train_fold{}.ix[:,0]'.format(j,j))
    
# Seperate All Features Out Of The Training Set {1-10} 
    print ('train_fold{} = train_fold{}.ix[{}:,1:]'.format(j,j, 0))
    
# Drop 1/10th Of The Excluded Training Set Which Is Used As The Test Data For Cross Validation 
    print ('train_fold{} = train_fold{}.drop(train_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1))
    
# Now Drop 1/10th Of The Excluded Target Set To Match The Shape Of The Training Set
    print ('target_fold{} = target_fold{}.drop(target_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1), "\n")

# Iterate By 1. This Will Stop At The Number Of Training KFolds We Have (e.g. 10 Folds)
    j += 1

train_fold1 = train
test_fold1 = train.ix[0:60, 1:]
target_fold1 = train_fold1.ix[:,0]
train_fold1 = train_fold1.ix[0:,1:]
train_fold1 = train_fold1.drop(train_fold1.index[0:60])
target_fold1 = target_fold1.drop(target_fold1.index[0:60]) 

train_fold2 = train
test_fold2 = train.ix[60:120, 1:]
target_fold2 = train_fold2.ix[:,0]
train_fold2 = train_fold2.ix[0:,1:]
train_fold2 = train_fold2.drop(train_fold2.index[60:120])
target_fold2 = target_fold2.drop(target_fold2.index[60:120]) 

train_fold3 = train
test_fold3 = train.ix[120:180, 1:]
target_fold3 = train_fold3.ix[:,0]
train_fold3 = train_fold3.ix[0:,1:]
train_fold3 = train_fold3.drop(train_fold3.index[120:180])
target_fold3 = target_fold3.drop(target_fold3.index[120:180]) 

train_fold4 = train
test_fold4 = train.ix[180:240, 1:]
target_fold4 = train_fold4.ix[:,0]
train_fold4 = train_fold4.ix[0:,1:]
train_fold4 = train_fold4.drop(train_fold4.index[180:240])
target_fold4 = target_fold4.drop(target_fold4.index[180:240]) 

train_fold5 = tr

In [34]:
train_fold1 = train
test_fold1 = train.ix[0:60, 1:]
target_fold1 = train_fold1.ix[:,0]
train_fold1 = train_fold1.ix[0:,1:]
train_fold1 = train_fold1.drop(train_fold1.index[0:60])
target_fold1 = target_fold1.drop(target_fold1.index[0:60]) 

train_fold2 = train
test_fold2 = train.ix[60:120, 1:]
target_fold2 = train_fold2.ix[:,0]
train_fold2 = train_fold2.ix[0:,1:]
train_fold2 = train_fold2.drop(train_fold2.index[60:120])
target_fold2 = target_fold2.drop(target_fold2.index[60:120]) 

train_fold3 = train
test_fold3 = train.ix[120:180, 1:]
target_fold3 = train_fold3.ix[:,0]
train_fold3 = train_fold3.ix[0:,1:]
train_fold3 = train_fold3.drop(train_fold3.index[120:180])
target_fold3 = target_fold3.drop(target_fold3.index[120:180]) 

train_fold4 = train
test_fold4 = train.ix[180:240, 1:]
target_fold4 = train_fold4.ix[:,0]
train_fold4 = train_fold4.ix[0:,1:]
train_fold4 = train_fold4.drop(train_fold4.index[180:240])
target_fold4 = target_fold4.drop(target_fold4.index[180:240]) 

train_fold5 = train
test_fold5 = train.ix[240:300, 1:]
target_fold5 = train_fold5.ix[:,0]
train_fold5 = train_fold5.ix[0:,1:]
train_fold5 = train_fold5.drop(train_fold5.index[240:300])
target_fold5 = target_fold5.drop(target_fold5.index[240:300]) 

train_fold6 = train
test_fold6 = train.ix[300:360, 1:]
target_fold6 = train_fold6.ix[:,0]
train_fold6 = train_fold6.ix[0:,1:]
train_fold6 = train_fold6.drop(train_fold6.index[300:360])
target_fold6 = target_fold6.drop(target_fold6.index[300:360]) 

train_fold7 = train
test_fold7 = train.ix[360:419, 1:]
target_fold7 = train_fold7.ix[:,0]
train_fold7 = train_fold7.ix[0:,1:]
train_fold7 = train_fold7.drop(train_fold7.index[360:419])
target_fold7 = target_fold7.drop(target_fold7.index[360:419]) 

train_fold8 = train
test_fold8 = train.ix[419:478, 1:]
target_fold8 = train_fold8.ix[:,0]
train_fold8 = train_fold8.ix[0:,1:]
train_fold8 = train_fold8.drop(train_fold8.index[419:478])
target_fold8 = target_fold8.drop(target_fold8.index[419:478]) 

train_fold9 = train
test_fold9 = train.ix[478:537, 1:]
target_fold9 = train_fold9.ix[:,0]
train_fold9 = train_fold9.ix[0:,1:]
train_fold9 = train_fold9.drop(train_fold9.index[478:537])
target_fold9 = target_fold9.drop(target_fold9.index[478:537]) 

train_fold10 = train
test_fold10 = train.ix[537:596, 1:]
target_fold10 = train_fold10.ix[:,0]
train_fold10 = train_fold10.ix[0:,1:]
train_fold10 = train_fold10.drop(train_fold10.index[537:596])
target_fold10 = target_fold10.drop(target_fold10.index[537:596]) 

train_fold11 = train
test_fold11 = train.ix[596:655, 1:]
target_fold11 = train_fold11.ix[:,0]
train_fold11 = train_fold11.ix[0:,1:]
train_fold11 = train_fold11.drop(train_fold11.index[596:655])
target_fold11 = target_fold11.drop(target_fold11.index[596:655]) 

train_fold12 = train
test_fold12 = train.ix[655:714, 1:]
target_fold12 = train_fold12.ix[:,0]
train_fold12 = train_fold12.ix[0:,1:]
train_fold12 = train_fold12.drop(train_fold12.index[655:714])
target_fold12 = target_fold12.drop(target_fold12.index[655:714]) 

train_fold13 = train
test_fold13 = train.ix[714:773, 1:]
target_fold13 = train_fold13.ix[:,0]
train_fold13 = train_fold13.ix[0:,1:]
train_fold13 = train_fold13.drop(train_fold13.index[714:773])
target_fold13 = target_fold13.drop(target_fold13.index[714:773]) 

train_fold14 = train
test_fold14 = train.ix[773:832, 1:]
target_fold14 = train_fold14.ix[:,0]
train_fold14 = train_fold14.ix[0:,1:]
train_fold14 = train_fold14.drop(train_fold14.index[773:832])
target_fold14 = target_fold14.drop(target_fold14.index[773:832]) 

train_fold15 = train
test_fold15 = train.ix[832:891, 1:]
target_fold15 = train_fold15.ix[:,0]
train_fold15 = train_fold15.ix[0:,1:]
train_fold15 = train_fold15.drop(train_fold15.index[832:891])
target_fold15 = target_fold15.drop(target_fold15.index[832:891]) 




##### Shape Of Each Fold

In [35]:
"""This Code Only Outputs The Code To Look At The Shape Of Each Fold: Helpful To Make Sure Our \
Training Folds, Target Folds, and Testing Folds Are The Same Sizes Throughout"""
# for j in [1,2,3,4,5]:
#     print("print(train_fold{}.shape)".format(j))
#     print("print(target_fold{}.shape)".format(j))
#     print("print(test_fold{}.shape)".format(j), "\n")

'This Code Only Outputs The Code To Look At The Shape Of Each Fold: Helpful To Make Sure Our Training Folds, Target Folds, and Testing Folds Are The Same Sizes Throughout'

In [36]:
print(train_fold1.shape)
print(target_fold1.shape)
print(test_fold1.shape) 

print(train_fold2.shape)
print(target_fold2.shape)
print(test_fold2.shape) 

print(train_fold3.shape)
print(target_fold3.shape)
print(test_fold3.shape) 

print(train_fold4.shape)
print(target_fold4.shape)
print(test_fold4.shape) 

print(train_fold5.shape)
print(target_fold5.shape)
print(test_fold5.shape) 

(831, 16)
(831,)
(61, 16)
(831, 16)
(831,)
(61, 16)
(831, 16)
(831,)
(61, 16)
(831, 16)
(831,)
(61, 16)
(831, 16)
(831,)
(61, 16)


### Part 3
- Use Model M1 to train on each train fold i and predict the excluded fold and store the values back into M1 for its fold ID
- Do the same with M2

In [37]:
"""Instantiate Models"""
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

m1 = KNeighborsClassifier(n_neighbors = 9)
m2 = GradientBoostingClassifier(n_estimators = 79)
m3 = LogisticRegression(C = 1, max_iter =  5)
m4 = RandomForestClassifier(n_estimators = 42)
m5 = svm.SVC(kernel='linear', C=2,gamma='auto', probability=True)
m6 = GaussianNB()

#### Create An Empty List That Will Join The Outputs From Each Fold And Store Them Into M1 Column

In [38]:
"""Create A List Of The Folds. This Will Make Looping Through The Folds Quicker"""
train_folds = [train_fold1, train_fold2, train_fold3, train_fold4, train_fold5, \
               train_fold6, train_fold7, train_fold8, train_fold9, train_fold10, \
               train_fold11, train_fold12, train_fold13, train_fold14, train_fold15]
target_folds = [target_fold1, target_fold2, target_fold3, target_fold4, target_fold5, \
                target_fold6, target_fold7, target_fold8, target_fold9, target_fold10, \
               target_fold11, target_fold12, target_fold13, target_fold14, target_fold15]
test_folds = [test_fold1, test_fold2, test_fold3, test_fold4, test_fold5, \
              test_fold6, test_fold7, test_fold8, test_fold9, test_fold10, \
             test_fold11, test_fold12, test_fold13, test_fold14, test_fold15]

In [39]:
"""Train Data On First Model"""

"""Create An Empty List That Will Join The Predicted Outputs From Each Fold And Store Them Into M1 Column"""
y_pred1 = []
for i, j in zip(train_folds, target_folds):
    m1.fit(i, j)
for k in test_folds:
    target_pred = m1.predict(k)
    y_pred1.append(target_pred) 
    
y_pred1 = np.concatenate(y_pred1, axis=0)
y_pred1 = pd.DataFrame(y_pred1)
train_meta["M1"] = y_pred1

In [40]:
"""Train Data On Second Model"""

"""Create An Empty List That Will Join The Outputs From Each Fold And Store Them Into M2 Column"""
y_pred2 = []
for i, j in zip(train_folds, target_folds):
    m2.fit(i, j)
for k in test_folds:
    target_pred2 = m2.predict(k)
    y_pred2.append(target_pred2) 
    
y_pred2 = np.concatenate(y_pred2, axis=0)
y_pred2 = pd.DataFrame(y_pred2)
train_meta["M2"] = y_pred2

In [41]:
"""Train Data On Third Model"""
y_pred3 = []
for i, j in zip(train_folds, target_folds):
    m3.fit(i, j)
for k in test_folds:
    target_pred3 = m3.predict(k)
    y_pred3.append(target_pred3) 
    
y_pred3 = np.concatenate(y_pred3, axis=0)
y_pred3 = pd.DataFrame(y_pred3)
train_meta["M3"] = y_pred3

In [42]:
"""Train Data On 4th Model"""
y_pred4 = []
for i, j in zip(train_folds, target_folds):
    m4.fit(i, j)
for k in test_folds:
    target_pred4 = m4.predict(k)
    y_pred4.append(target_pred4) 
    
y_pred4 = np.concatenate(y_pred4, axis=0)
y_pred4 = pd.DataFrame(y_pred4)
train_meta["M4"] = y_pred4

In [43]:
"""Train Data On 5th Model"""
y_pred5 = []
for i, j in zip(train_folds, target_folds):
    m5.fit(i, j)
for k in test_folds:
    target_pred5 = m5.predict(k)
    y_pred5.append(target_pred5) 
    
y_pred5 = np.concatenate(y_pred5, axis=0)
y_pred5 = pd.DataFrame(y_pred5)
train_meta["M5"] = y_pred5

In [44]:
"""Train Data On 6th Model"""
y_pred6 = []
for i, j in zip(train_folds, target_folds):
    m6.fit(i, j)
for k in test_folds:
    target_pred6 = m6.predict(k)
    y_pred6.append(target_pred6) 
    
y_pred6 = np.concatenate(y_pred6, axis=0)
y_pred6 = pd.DataFrame(y_pred6)
train_meta["M6"] = y_pred6

### Part 4

Fit m1 Model to train_meta
- Put new predictions into train_meta m1 column
Fit m2 Model to train_meta
- Put new predictions into train_meta m2 column

In [45]:
train_meta

,Survived,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,...,Age_Young,Fare_1,Pclass_2,Pclass_3,M1,M2,M3,M4,M5,M6
0,0.0,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,1.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.166667,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.333333,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


### Part 5
- Predict m1 on test_meta
- Predict m2 on test_meta

Part 4
Use another model (S) and use train_meta to make final predictions on test_meta 

In [46]:
train_meta.head()

,Survived,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,...,Age_Young,Fare_1,Pclass_2,Pclass_3,M1,M2,M3,M4,M5,M6
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Store Our New Targets (M1, M2, Survived) From Our Training Set Into New Target Variables
    - Delete Them Off Training Set
    - These Will Be Used To Fit Out Models To Train On Our Test Data
    - Before Deleting Them Off, Make A Copy For When We Train On Our S Model

In [47]:
train_meta_Class = train_meta["Survived"]
train_meta_M1 = train_meta["M1"]
train_meta_M2 = train_meta["M2"]
train_meta_M3 = train_meta["M3"]
train_meta_M4 = train_meta["M4"]
train_meta_M5 = train_meta["M5"]
train_meta_M6 = train_meta["M6"]

del train_meta["Survived"]

"""Create A New Training Set For Predictions Of Test Data With M1, M2, M3 Columns"""
training = pd.DataFrame.copy(train_meta)
del train_meta["M1"], train_meta["M2"], train_meta["M3"], train_meta["M4"], train_meta["M5"], train_meta["M6"]

In [48]:
train_meta

,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,SibSp_Has1Family,SibSp_HasMultipleFamily,Age_Young,Fare_1,Pclass_2,Pclass_3
0,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.166667,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
8,0.333333,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [49]:
"""Train Data On Third Model"""
m1.fit(train_meta, train_meta_M1)
target_pred_M1 = m1.predict(test_meta)

m2.fit(train_meta, train_meta_M2)
target_pred_M2 = m2.predict(test_meta)

m3.fit(train_meta, train_meta_M3)
target_pred_M3 = m3.predict(test_meta)

m4.fit(train_meta, train_meta_M4)
target_pred_M4 = m4.predict(test_meta)

m5.fit(train_meta, train_meta_M5)
target_pred_M5 = m5.predict(test_meta)

m6.fit(train_meta, train_meta_M6)
target_pred_M6 = m6.predict(test_meta)

test_meta["M1"] = target_pred_M1
test_meta["M2"] = target_pred_M2
test_meta["M3"] = target_pred_M3
test_meta["M4"] = target_pred_M4
test_meta["M5"] = target_pred_M5
test_meta["M6"] = target_pred_M6

In [50]:
test_meta.head()

,Parch,Sex_1,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Cabin_Middle Class,Cabin_Upper Class,...,Age_Young,Fare_1,Pclass_2,Pclass_3,M1,M2,M3,M4,M5,M6
0,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.111111,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


### Part 6 Predict On Actual Testing Set

In [51]:
s = VotingClassifier(estimators=[('m1', m1), ('m2', m2), ('m3', m3), \
                                 ('m4', m4), ('m5', m5), ('m6', m6)], voting='soft')
s.fit(training, train_meta_Class)
target_pred_Class = s.predict(test_meta)

test_meta["Survived"] = target_pred_Class

In [52]:
predictions = pd.DataFrame(ID)
def predict(predictions):
    predictions["Survived"] = target_pred_Class
    predictions = predictions.astype(int)
    return predictions

answers = predict(predictions).astype(int)

In [53]:
answers.to_csv('answers1.csv',index = False)

In [54]:
answers

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
